### Inicio Spark Session
En esta primera celda importamos y configuramos todo lo necesario e iniciamos nuestra sesión de Spark

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._


//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//Circe para trabajar con JSON
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._


//Sttp para realizar request
import $ivy.`com.softwaremill.sttp.client::async-http-client-backend-monix:2.0.0-RC2`
import sttp.client.quick.{quickRequest, UriContext, backend}
import sttp.client.asynchttpclient.monix._
import sttp.client.basicRequest

//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  



import org.apache.spark.sql._

import org.apache.spark.sql.functions._


//Reducir numero logs

import org.apache.log4j.{Level, Logger}

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._


//Sttp para realizar request

import $ivy.$                                                                        

import sttp.client.quick.{quickRequest, UriContext, backend}

import sttp.client.asynchttpclient.monix._

import sttp.client.basicRequest

//For adding extra dependenies

import $ivy.$                               

//Plotly

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

//Iniciamos session de Spark

spark: SparkSession = org.apache.spark.sql.SparkSession@584f813f
import spark.implicits._

In [2]:
val dfRegiones = spark.read
            .parquet("../../data/parquet/dsBalanceNacionalPorRegionesMensual11-24.parquet")

parquet at cmd1.sc:2

1 / 1

dfRegiones: DataFrame = [Familia: string, Tipo: string ... 6 more fields]

In [3]:
//Quitar peninsular para quedarnos solo con comunidades
val dataRegiones = dfRegiones
    .filter($"Region" !== "Peninsular")
    //.filter($"Compuesto" === false) //Fuente unica, no suma de energias

dataRegiones: Dataset[Row] = [Familia: string, Tipo: string ... 6 more fields]

In [6]:
import org.apache.spark.sql.expressions.Window

// Agrupar por "Region" y "Tipo", y calcular la suma de "Valor"

val año = "2024"

val sumaEnergiaPorRegionYTipo = dataRegiones
    .filter($"Compuesto" === false) 
    .filter($"Familia" =!= "Demanda")
    .filter(date_format(col("FechaCompleta"), "yyyy") === año)
    .groupBy($"Region", $"Tipo")
    .agg(sum("Valor").as("ValorTotal"))
    .withColumn("Valor total por tipo (millones MWh)", round(col("ValorTotal") / 1e6, 2))

val sumaEnergiaPorRegion = dataRegiones
    .filter($"Compuesto" === false) 
    .filter($"Familia" =!= "Demanda")
    .filter(date_format(col("FechaCompleta"), "yyyy") === año) 
    .groupBy($"Region")
    .agg(sum("Valor").as("ValorTotalPorRegion"))
    .withColumn("Valor total Region (millones MWh)", round(col("ValorTotalPorRegion") / 1e6, 2))

// Definir una ventana para particionar por "Region" y ordenar "ValorTotal" descendente
val windowSpec = Window.partitionBy("Region").orderBy(desc("ValorTotal"))

// Añadir columna de ranking para cada tipo de energía en cada región
val dfWithRank = sumaEnergiaPorRegionYTipo.withColumn("rank", row_number().over(windowSpec))

// Filtrar el top 1 de cada región
val topEnergiaPorRegion = dfWithRank.filter($"rank" === 1).drop("rank")

// Hacemos un join para juntar las dos columnas
val energiaTotalConTopEnergia = sumaEnergiaPorRegion
    .join(topEnergiaPorRegion, Seq("Region"), "inner") // Unimos ambos DataFrames por la columna "Region"
    .orderBy(desc("ValorTotalPorRegion"))
    .select(
    col("Region"),
    col("Valor total Region (millones MWh)").as("Energía Total por Región (millones MWh)"),
    col("Tipo").as("Fuente de Energía Predominante"),
    col("Valor total por tipo (millones MWh)").as("Energía de Fuente Predominante (millones MWh)")
    )


energiaTotalConTopEnergia.show()    

run at ThreadPoolExecutor.java:1149

16 / 16

run at ThreadPoolExecutor.java:1149

200 / 200

show at cmd5.sc:44

...

200 / 200

show at cmd5.sc:44

200 / 200

+--------------------+---------------------------------------+------------------------------+---------------------------------------------+
|              Region|Energía Total por Región (millones MWh)|Fuente de Energía Predominante|Energía de Fuente Predominante (millones MWh)|
+--------------------+---------------------------------------+------------------------------+---------------------------------------------+
|            Cataluña|                                  18.76|                       Nuclear|                                        11.07|
|         Extremadura|                                  16.29|                       Nuclear|                                         7.08|
|  Castilla la Mancha|                                  15.45|            Solar fotovoltaica|                                         6.19|
|           Andalucía|                                  15.36|            Solar fotovoltaica|                                         5.77|
|     Castilla y Leo

import org.apache.spark.sql.expressions.Window

// Agrupar por "Region" y "Tipo", y calcular la suma de "Valor"


año: String = "2024"
sumaEnergiaPorRegionYTipo: DataFrame = [Region: string, Tipo: string ... 2 more fields]
sumaEnergiaPorRegion: DataFrame = [Region: string, ValorTotalPorRegion: double ... 1 more field]
windowSpec: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@841812e
dfWithRank: DataFrame = [Region: string, Tipo: string ... 3 more fields]
topEnergiaPorRegion: DataFrame = [Region: string, Tipo: string ... 2 more fields]
energiaTotalConTopEnergia: DataFrame = [Region: string, Energía Total por Región (millones MWh): double ... 2 more fields]

In [7]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

// Definir el año a filtrar
val año = "2024"

// Aplicar los filtros iniciales
val dfFiltrado = dataRegiones
    .filter($"Compuesto" === false) 
    .filter($"Familia" =!= "Demanda")
    .filter(date_format(col("FechaCompleta"), "yyyy") === año)

// Agrupar por "Region" y "Tipo", y calcular la suma de "Valor" para cada tipo de energía en cada región
val sumaEnergiaPorRegionYTipo = dfFiltrado
    .groupBy($"Region", $"Tipo")
    .agg(sum("Valor").as("ValorTotal"))

// Definir una ventana para obtener el tipo de energía con la mayor producción para cada región
val windowSpec = Window.partitionBy("Region").orderBy(desc("ValorTotal"))

// Añadir columna de ranking y seleccionar el top 1 por región
val topEnergiaPorRegion = sumaEnergiaPorRegionYTipo
    .withColumn("rank", row_number().over(windowSpec))
    .filter($"rank" === 1)
    .drop("rank")

// Calcular la suma de la energía total por cada región
val sumaEnergiaPorRegion = sumaEnergiaPorRegionYTipo
    .groupBy($"Region")
    .agg(sum("ValorTotal").as("ValorTotalPorRegion"))
    .withColumn("Valor total Region (millones MWh)", round(col("ValorTotalPorRegion") / 1e6, 2))

// Hacemos un join para combinar los datos de la energía total y la fuente predominante
val energiaTotalConTopEnergia = sumaEnergiaPorRegion
    .join(topEnergiaPorRegion, Seq("Region"), "inner") // Unimos ambos DataFrames por la columna "Region"
    .select(
        col("Region"),
        col("Valor total Region (millones MWh)").as("Energía Total por Región (millones MWh)"),
        col("Tipo").as("Fuente de Energía Predominante"),
        round(col("ValorTotal") / 1e6, 2).as("Energía de Fuente Predominante (millones MWh)")
    )
    .orderBy(desc("Energía Total por Región (millones MWh)"))

// Mostrar el resultado
energiaTotalConTopEnergia.show()


run at ThreadPoolExecutor.java:1149

16 / 16

run at ThreadPoolExecutor.java:1149

200 / 200

run at ThreadPoolExecutor.java:1149

...

200 / 200

show at cmd6.sc:45

200 / 200

+--------------------+---------------------------------------+------------------------------+---------------------------------------------+
|              Region|Energía Total por Región (millones MWh)|Fuente de Energía Predominante|Energía de Fuente Predominante (millones MWh)|
+--------------------+---------------------------------------+------------------------------+---------------------------------------------+
|            Cataluña|                                  18.76|                       Nuclear|                                        11.07|
|         Extremadura|                                  16.29|                       Nuclear|                                         7.08|
|  Castilla la Mancha|                                  15.45|            Solar fotovoltaica|                                         6.19|
|           Andalucía|                                  15.36|            Solar fotovoltaica|                                         5.77|
|     Castilla y Leo

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.functions._

// Definir el año a filtrar

año: String = "2024"
dfFiltrado: Dataset[Row] = [Familia: string, Tipo: string ... 6 more fields]
sumaEnergiaPorRegionYTipo: DataFrame = [Region: string, Tipo: string ... 1 more field]
windowSpec: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1d2220ff
topEnergiaPorRegion: DataFrame = [Region: string, Tipo: string ... 1 more field]
sumaEnergiaPorRegion: DataFrame = [Region: string, ValorTotalPorRegion: double ... 1 more field]
energiaTotalConTopEnergia: Dataset[Row] = [Region: string, Energía Total por Región (millones MWh): double ... 2 more fields]

In [9]:
sumaEnergiaPorRegionYTipo
    .withColumn("rank", row_number().over(windowSpec))
    .show()

show at cmd8.sc:2

16 / 16

show at cmd8.sc:2

200 / 200

show at cmd8.sc:2

...

show at cmd8.sc:2

7 / 7

show at cmd8.sc:2

5 / 5

+--------------------+--------------------+------------------+----+
|              Region|                Tipo|        ValorTotal|rank|
+--------------------+--------------------+------------------+----+
|Comunidad Valenciana|             Nuclear|       3447199.037|   1|
|Comunidad Valenciana|  Turbinación bombeo|       1151945.338|   2|
|Comunidad Valenciana|              Eólica|        979153.284|   3|
|Comunidad Valenciana|     Ciclo combinado|        959049.021|   4|
|Comunidad Valenciana|        Cogeneración|        453825.985|   5|
|Comunidad Valenciana|  Solar fotovoltaica|        390546.888|   6|
|Comunidad Valenciana|          Hidráulica|         338831.28|   7|
|Comunidad Valenciana|       Solar térmica|         58696.936|   8|
|Comunidad Valenciana|    Otras renovables|         14203.804|   9|
|Comunidad Valenciana|Residuos no renov...|          8974.212|  10|
|Comunidad Valenciana|          Fuel + Gas|               0.0|  11|
|              Madrid|        Cogeneración|     